In [1]:
import os
import numpy as np
import pandas as pd
import pickle

from bqplot import LinearScale, Axis, Lines, Scatter, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML, Output

In [2]:
ddir = '/epyc/projects/sso-lc/notebooks/aug_29_2019'
filename = 't.csv'
d = pd.read_csv(os.path.join(ddir, filename))

In [3]:
both = d.query('period > 0')

In [4]:
def plot_scatter(x=[], y=[], zid=[], color='red', filt=''):
    '''Create and return Scatter plot'''
    #TODO: tooltip format with all data  
    tooltip = bq.Tooltip(fields=['x', 'y', 'zid'], formats=['.2f', '.2f', ''],
                        labels=['fit_period', 'lcd_period', 'ztfname'])
    sc_x = bq.LinearScale(min=0, max=50)
    sc_y = bq.LinearScale(min=0, max=50)
    
    scatt = bq.Scatter(
            scales={'x': sc_x, 'y': sc_y},
            tooltip=tooltip,
            tooltip_style={'opacity': 0.5},
            interactions={'hover': 'tooltip'},
            unhovered_style={'opacity': 0.5},
            selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
            unselected_style={'opacity': 0.5},
            display_legend=False)
    scatt.colors = [color]
    scatt.label = filt
    if ((y != [])):
        scatt.x = x
        scatt.y = y
    #scatt.on_element_click(display_info)

    return scatt

In [5]:
table = Output(layout={'border': '1px solid black'})

scales = {'x': LinearScale(min=0, max=60.), 'y': LinearScale(min=0, max=60.)}

scatt = Scatter(x=d["fit_period"], y=d['period'], scales=scales)

ax_x = Axis(scale=scales['x'], tick_format='0.0f', label='Fit period')
ax_y = Axis(scale=scales['y'], orientation='vertical', tick_format='0.0f', label='ALCDEF period')

selector = BrushIntervalSelector(scale=scales['x'], orientation='horizontal', marks=[scatt])
#sel_date = BrushSelector(x_scale=scales['x'], y_scale=scales['y'],marks=[lc2_index], color="green")
#intsel_date = BrushSelector(x_scale=dt_x_index, y_scale=lin_y2, marks=[lc2_index])

def interval_change_callback(change):
    #print (change)
    #db_date.value = str(change.new)
    index = change.new
#    print (df)
    with table:
        table.clear_output()
        if (index != None):
            display(d.iloc[index])

## Notice here that we call the observe on the Mark lc2_index rather than on the selector intsel_date
scatt.observe(interval_change_callback, names=['selected',])

fig = Figure(marks=[scatt], axes=[ax_x, ax_y],
                       title='Period Plot', interaction=selector)

VBox([fig, table])